## The object of this project is to explore, segment, and cluster the neighborhoods in the city of Toronto. From the analysis, we can obtain the insight that what regions in Toronto are similar to each other and different from the rest in terms of some common or distinguishable features. 

# Note: since view of Folium map rendering is not working in Github, please copy the url of this file to https://nbviewer.jupyter.org/ to view them.

### The project includes three parts:
#### - Part 1 (in a separated Notebook) involves scraping and cleasing the data of the basic Toronto neighborhood information. 
#### - Part 2 (in a separated Notebook) continues with the cleaned neighborhood data and uses Foursquare API to retrieve more details around each neighborhood, which are considered features and needed for further analysis. 
#### - In Part 3, the features are transformed so that segmenting and clustering can be applied. Folium map module is used here for geographical visualization. K-means clustering through Scikit-learn is used here for the model, and insight is found and analyzed in this section.  

## Part 3: explore and cluster the neighbourhoods and find insights

In [1]:
# import necessary modules 
import numpy as np
import pandas as pd

import requests

# import folium maps module
#!conda install -c conda-forge folium=0.5.0 --yes  
import folium 

# import modules needed for geo location conversion
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

Read the cleaned and grouped data with coordinates organized from Part 1 & 2 

In [2]:
url_cleaned_data = 'https://raw.githubusercontent.com/liangyiteng888/Coursera_Capstone/master/Toronto_Neighborhood_with_Coordinates.csv'
df_neighbor_toronto = pd.read_csv(url_cleaned_data, index_col = 0)
#help(pd.read_csv)

Uncomment the selection part if only wanting to Work on boroughs that contain the word 'Toronto'. Extract matching rows from the dataframe.

In [3]:
df_neighbor_toronto = df_neighbor_toronto[['Toronto' in i for i in df_neighbor_toronto['Borough']]]
df_neighbor_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Reorganize the dataframe so that same Borough rows gather together 

In [4]:
df_neighbor_toronto = df_neighbor_toronto.sort_values('Borough').reset_index(drop=True)
df_neighbor_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Create a map of Toronto with neighborhoods superimposed on top. Use **Folium** map rendering library for the map creation and manipulation.

In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
# create map of Toronto  using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighbor_toronto['Latitude'], df_neighbor_toronto['Longitude'], df_neighbor_toronto['Borough'], df_neighbor_toronto['Neighborhood']):
    label = '{} ({})'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2' # your Foursquare ID
CLIENT_SECRET = 'BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE' # your Foursquare Secret
VERSION = '20200220' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2
CLIENT_SECRET:BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE


In [8]:
neighborhood_latitude = df_neighbor_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_neighbor_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_neighbor_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Annex, North Midtown, Yorkville are 43.6727097, -79.40567840000001.


#### Now, get the top 100 venues that are in The Annex, North Midtown, Yorkville within a radius of 500 meters.
First, let's create the GET request URL. Name your URL **url**.

In [9]:
LIMIT= 100
radius = 500
url =  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2&client_secret=BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE&v=20200220&ll=43.6727097,-79.40567840000001&radius=500&limit=100'

In [10]:
results = requests.get(url).json()

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
#print(venues)
nearby_venues = json_normalize(venues) # flatten JSON
#print(nearby_venues.head())
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ezra's Pound,Café,43.675153,-79.405858
1,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249
2,Rose & Sons,American Restaurant,43.675668,-79.403617
3,Live Organic Food Bar,Vegetarian / Vegan Restaurant,43.675053,-79.406715
4,Fet Zun,Middle Eastern Restaurant,43.675147,-79.406346


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


### Part 3.1 Explore Neighborhoods in Toronto

Create a function to repeat the same process to all the neighborhoods in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
Toronto_venues = getNearbyVenues(names=df_neighbor_toronto['Neighborhood'],
                                   latitudes=df_neighbor_toronto['Latitude'],
                                   longitudes=df_neighbor_toronto['Longitude']
                                  )

The Annex, North Midtown, Yorkville
Forest Hill North, Forest Hill West
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Harbourfront East, Toronto Islands, Union Station
Harbord, University of Toronto
Commerce Court, Victoria Hotel
Design Exchange, Toronto Dominion Centre
Queen's Park
Adelaide, King, Richmond
Central Bay Street
Harbourfront
St. James Town
Ryerson, Garden District
Chinatown, Grange Park, Kensington Market
Church and Wellesley
Cabbagetown, St. James Town
Rosedale
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Berczy Park
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
The Beaches
The Beaches West, India Bazaar
The Danforth West, Riverdale
Studio District
Business Reply Mail Processing Centre 969 Eastern
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, 

#### Drop records where the 'Venue Category' is indicated as 'Neighborhood'

In [16]:
print(Toronto_venues.shape)
Toronto_venues = Toronto_venues.drop(Toronto_venues[['Neighborhood' in i for i in Toronto_venues['Venue Category']]].index).reset_index(drop = True)

(1731, 7)


Show the number of venues obtained for each of the neighborhood

In [17]:
Toronto_venues_count = Toronto_venues.groupby('Neighborhood')['Venue'].count().reset_index(name = 'Number of venues')
Toronto_venues_count

,Neighborhood,Number of venues
0,"Adelaide, King, Richmond",99
1,Berczy Park,57
2,"Brockton, Exhibition Place, Parkdale Village",23
3,Business Reply Mail Processing Centre 969 Eastern,18
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",17
5,"Cabbagetown, St. James Town",49
6,Central Bay Street,79
7,"Chinatown, Grange Park, Kensington Market",87
8,Christie,18
9,Church and Wellesley,86


**Neighborhoods with venues fewer than 5 can negatively affect the clustering modelling. So, let's remove these Neighborhoods from the list.**

In [18]:
Neighborhoods_not_enough_venues = Toronto_venues_count[Toronto_venues_count['Number of venues'] >=10]['Neighborhood'].values
Toronto_venues_cleaned = Toronto_venues[Toronto_venues['Neighborhood'].isin(Neighborhoods_not_enough_venues)]
Toronto_venues_cleaned.groupby('Neighborhood')['Venue'].count().reset_index(name = 'Number of venues')

,Neighborhood,Number of venues
0,"Adelaide, King, Richmond",99
1,Berczy Park,57
2,"Brockton, Exhibition Place, Parkdale Village",23
3,Business Reply Mail Processing Centre 969 Eastern,18
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",17
5,"Cabbagetown, St. James Town",49
6,Central Bay Street,79
7,"Chinatown, Grange Park, Kensington Market",87
8,Christie,18
9,Church and Wellesley,86


In [19]:
print('There are {} uniques categories.'.format(len(Toronto_venues_cleaned['Venue Category'].unique())))

There are 234 uniques categories.


### Part 3.2 Transform the venue categories for each neighborhood to be applicable for clustering 

Used one hot encoding to transform the venue categories into 0/1 numbers needed as input for the clustering modelling

In [20]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues_cleaned[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues_cleaned['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
#print(fixed_columns)
print(Toronto_onehot.shape)
Toronto_onehot.head()

(1698, 235)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

(32, 235)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.000000,0.000000,...,0.00000,0.000000,0.020202,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,...,0.00000,0.000000,0.012658,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.034483,0.000000,0.057471,0.011494,0.000000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.011628,0.000000,0.011628


#### Print each neighborhood along with the top 3 most common venues

In [22]:
num_top_venues = 3

for row in Toronto_grouped['Neighborhood']:
    print("----"+row+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == row].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.04
2   Restaurant  0.04


----Berczy Park----
               venue  freq
0        Coffee Shop  0.07
1  French Restaurant  0.04
2           Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.06
2               Brewery  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.08
1  Pizza Place  0.06
2       Bakery  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop 

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Gym,Bookstore,Concert Hall,Cosmetics Shop
1,Berczy Park,Coffee Shop,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,French Restaurant,Cocktail Bar,Café,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Café,Bakery,Coffee Shop,Breakfast Spot,Gym,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Sculpture Garden


### Part 3.3 Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = 'k-means++', n_clusters=kclusters, random_state=0, n_init =12).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([3, 3, 3, 3, 1, 5, 2, 0, 6, 3, 2, 5, 5, 2, 0, 2, 3, 2, 2, 0, 0, 3,
       4, 2, 3, 3, 3, 3, 3, 5, 5, 5])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
                               
toronto_merged = df_neighbor_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,5.0,Coffee Shop,Café,Sandwich Place,Pizza Place,Pub,BBQ Joint,History Museum,Middle Eastern Restaurant,Liquor Store,American Restaurant
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3.0,Clothing Store,Coffee Shop,Sporting Goods Shop,Diner,Mexican Restaurant,Miscellaneous Shop,Dessert Shop,Park,Bagel Shop,Fast Food Restaurant


In [27]:
print('check if there is any N/A entries', toronto_merged.isnull().values.sum())
toronto_merged.dropna(inplace = True)

check if there is any N/A entries 77


In [28]:
print('check if there is any N/A entries', toronto_merged.isnull().values.sum())
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')

check if there is any N/A entries 0


#### Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) +')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.4 Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 0 - Asian towns

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5T,"Chinatown, Grange Park, Kensington Market",0,Bar,Café,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Comfort Food Restaurant
33,M6H,"Dovercourt Village, Dufferin",0,Bakery,Pharmacy,Recording Studio,Bus Stop,Brewery,Café,Supermarket,Music Venue,Bank,Gym / Fitness Center
34,M6J,"Little Portugal, Trinity",0,Bar,Coffee Shop,Asian Restaurant,Restaurant,Pizza Place,Bakery,Men's Store,Vietnamese Restaurant,Wine Bar,Café
36,M6P,"High Park, The Junction South",0,Thai Restaurant,Café,Mexican Restaurant,Bar,Park,Fast Food Restaurant,Bookstore,Cajun / Creole Restaurant,Flea Market,Speakeasy


#### Cluster 1 - Airport/ Boat Ports

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Sculpture Garden


#### Cluster 2 - Entertainment of coffee, cafe and clubs

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5J,"Harbourfront East, Toronto Islands, Union Station",2,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Italian Restaurant,Restaurant,Brewery,Scenic Lookout,Music Venue
11,M5L,"Commerce Court, Victoria Hotel",2,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Japanese Restaurant,Seafood Restaurant,Gastropub
12,M5K,"Design Exchange, Toronto Dominion Centre",2,Coffee Shop,Café,Restaurant,Hotel,Bar,Bakery,Gastropub,American Restaurant,Italian Restaurant,Japanese Restaurant
13,M7A,Queen's Park,2,Coffee Shop,Park,Burger Joint,Yoga Studio,Distribution Center,Portuguese Restaurant,Nightclub,Music Venue,Mexican Restaurant,Juice Bar
15,M5G,Central Bay Street,2,Coffee Shop,Sandwich Place,Italian Restaurant,Burger Joint,Café,Ice Cream Shop,Juice Bar,Japanese Restaurant,Thai Restaurant,Salad Place
16,M5A,Harbourfront,2,Coffee Shop,Café,Park,Bakery,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Theater,Event Space
24,M5X,"First Canadian Place, Underground city",2,Coffee Shop,Café,Restaurant,American Restaurant,Seafood Restaurant,Bar,Steakhouse,Deli / Bodega,Japanese Restaurant,Hotel


#### Cluster 3 - Mixed life zone with different cultures

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4R,North Toronto West,3,Clothing Store,Coffee Shop,Sporting Goods Shop,Diner,Mexican Restaurant,Miscellaneous Shop,Dessert Shop,Park,Bagel Shop,Fast Food Restaurant
10,M5S,"Harbord, University of Toronto",3,Café,Restaurant,Bakery,Bar,Bookstore,Japanese Restaurant,Italian Restaurant,Dessert Shop,Pub,Noodle House
14,M5H,"Adelaide, King, Richmond",3,Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Gym,Bookstore,Concert Hall,Cosmetics Shop
17,M5C,St. James Town,3,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Bakery,Italian Restaurant,Breakfast Spot,Beer Bar,Cosmetics Shop
18,M5B,"Ryerson, Garden District",3,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Theater,Plaza
20,M4Y,Church and Wellesley,3,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Fast Food Restaurant,Hotel,Pizza Place
23,M5W,Stn A PO Boxes 25 The Esplanade,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Seafood Restaurant,Beer Bar,Bakery,Lounge,Cocktail Bar
26,M5E,Berczy Park,3,Coffee Shop,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,French Restaurant,Cocktail Bar,Café,Beer Bar
31,M4M,Studio District,3,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
32,M7Y,Business Reply Mail Processing Centre 969 Eastern,3,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park


#### Cluster 4 - maybe an outlier

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M6R,"Parkdale, Roncesvalles",4,Breakfast Spot,Gift Shop,Restaurant,Dessert Shop,Eastern European Restaurant,Dog Run,Italian Restaurant,Bar,Movie Theater,Bookstore


#### Cluster 5 - mixed western and multiculture

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,"The Annex, North Midtown, Yorkville",5,Coffee Shop,Café,Sandwich Place,Pizza Place,Pub,BBQ Joint,History Museum,Middle Eastern Restaurant,Liquor Store,American Restaurant
5,M4S,Davisville,5,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Pizza Place,Gym,Indian Restaurant,Optical Shop
7,M4V,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",5,Coffee Shop,Pub,Bagel Shop,Fried Chicken Joint,Liquor Store,Restaurant,Sports Bar,Supermarket,Sushi Restaurant,Light Rail Station
21,M4X,"Cabbagetown, St. James Town",5,Coffee Shop,Pizza Place,Bakery,Café,Italian Restaurant,Pharmacy,Restaurant,Pub,Outdoor Sculpture,Ice Cream Shop
29,M4L,"The Beaches West, India Bazaar",5,Park,Sandwich Place,Pizza Place,Gym,Board Shop,Brewery,Burrito Place,Pub,Movie Theater,Sushi Restaurant
30,M4K,"The Danforth West, Riverdale",5,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Diner,Pub,Caribbean Restaurant,Bakery


#### Cluster 6 - maybe an outlier

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M6G,Christie,6,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Restaurant,Diner,Athletics & Sports,Baby Store


### By looking at the clusters, it is shown that the result is not optimized. Cluster 4 & 6 only have 1 candidate. Several modifications need to be done in the future for optimized solution:
- many restaurant types should be categorized into fewer typical ones, as some clusters have too many restaurants that might affect the clustering calculation 
- number of clusters have not been optimized, so there should have been one more step to evaluate and optimize the cluster number accuracy 